In [6]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
from dataclasses import dataclass
import urllib.request
import hashlib
import functools
#from nacl.signing import SigningKey

In [4]:
# block class
# each block has transactions
# output has a date attached --> how to check if output has been spent? add "spent" variable?

HashPointer = namedtuple('HashPointer', ['hash', 'pointer'])
Transaction = namedtuple('Transaction', ['inputs', 'outputs', 'age'])

Input = namedtuple('Input', ['previous_tx', 'index', 'public_key'])
SignedInput = namedtuple('SignedInput', ['input', 'signed_input'])

Output = namedtuple('Output', ['public_key_hash', 'value'])

class Block:
    def __init__(self, transactions, prev, nonce, pubkey_hash):
        self.transactions = transactions
        self.prev = prev
        self.nonce = nonce
        self.pubkey_hash = pubkey_hash
    
    def __repr__(self):
        return f'\nBlock(\n transaction: {self.transactions},\n nonce: {self.nonce},\n prev: {self.prev})'

def bytes_value(v):
    return bytes(str(v), encoding='utf-8')

def hash_value(v):
    return hashlib.sha256(bytes_value(v)).hexdigest()

def add_block(transactions, blockchain, nonce, pubkey_hash):
    prev_hash = hash_value(blockchain)
    prev = HashPointer(prev_hash, blockchain)
    new_block = Block(transactions, prev, nonce, pubkey_hash)
    
    return new_block, hash_value(new_block)

In [5]:
# peer class
# every peer has list of peers, do their receive_block to consider candidates
# take block with greater value

class Peer():
    transactions: dict[str, Transaction]
    current_block: Block
        
    @property
    def balance(self):
        running_bal = 0
        for key, tx in self.transactions:
            for output in tx.outputs:
                running_bal += output.value
        return running_bal
    
    def transaction_spent(self, transaction_hash):
        """Called by simulator to notify the peer that a transaction was spent!"""
        if transaction_hash in self.transactions:
            self.transactions.pop(transaction_hash)
    
    def receive_block_candidate(self, proposed_block):
        # validate block
        
        # calculate value of blocks
        # go thru transactions in the block and find oldest one. divide value / age (smaller => better)
        block_age = 0
        for tx in self.current_block.transactions:
            if block_age < tx.age:
                block_age = tx.age
                block_amount = functools.reduce(lambda a, b: a.value + b.value, tx.outputs)
        block_val = block_age // block_amount
            
        prop_block_age = 0
        for tx in proposed_block.transactions:
            if prop_block_age < tx.age:
                prop_block_age = tx.age
                prop_block_amount = functools.reduce(lambda a, b: a.value + b.value, tx.outputs)
        prop_block_val = prop_block_age // prop_block_amount
        
        
        # take block with smaller value
        if (block_val < prop_block_val):
            # add current_block to chain    
            
            # something here is funky. need to do shit with pubkey_hash 
            prev_hash = hash_value(current_block)
            prev = HashPointer(prev_hash, current_block)
            new_block = Block(self.current_block.transactions, prev, pubkey_hash)
            return new_block, hash_value(new_block)
    
        else:
            # add prop block to chain
            new_blockchain, final_hash = add_block(transaction, blockchain)
        # if vals are equal it's just going to go with the proposed one. this is probably fine
            
        
    def produce_block_candidate(self):
        pass
        


In [ ]:
# mining function
# difficulty is calculated by the product of the output of the transaction and the # of blocks where the 
# transaction output has not been spent

# the system we described wouldn't have to mine, so i'm not sure what to do with this
def mine_for_block(transactions, pubkey_hash, blockchain):
    nonce = 0
    while True:
        nonce = nonce + 1
        # 1. construct a block that contains the transaction
        new_blockchain, final_hash = add_block(transactions, blockchain, nonce, pubkey_hash)
        # 2. find a nonce such that the new block hashes to a number less than or equal to the difficulty
        if int(final_hash, 16) <= DIFFICULTY:
            print("Found a block! Final nonce:", nonce)
            return new_blockchain, final_hash

In [ ]:
class Simulator:
    peers: dict[str, Peer]
    transactions: dict[str, Transaction]

# simulator function 
# has a dictionary of peers --> look-up table so peers can interact
# record of transactions 
def run_simulator():
    # wallet addresses / keys : list of transactions that go to said address
    # dictionary of peer : address